## Module Imports

In [1]:
import numpy as np 
import pandas as pd 
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import os


## Dark Channel

In [2]:
def dark_channel(img,ks):
    J=np.min(img, axis=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(ks,ks))
    dark = cv2.erode(J,kernel)
    return dark

## Estimating the Transmission

In [3]:
# 2. Estimate Atmospheric Light
def estimate_atmospheric_light(img, dark_channel):
    flat_img = img.reshape(-1, 3)
    flat_dark = dark_channel.ravel()
    num_pixels = max(int(0.001 * len(flat_dark)), 1)  # 0.1% of total pixels
    top_indices = flat_dark.argsort()[-num_pixels:]
    atmospheric_light = np.mean(flat_img[top_indices], axis=0)
    return atmospheric_light

In [4]:
def estimate_transmission(img, atmospheric_light, omega=0.95, patch_size=15):
    norm_img = img / atmospheric_light
    dark_channel_trans = dark_channel(norm_img, patch_size)
    transmission = 1 - omega * dark_channel_trans
    return transmission

In [5]:
def refine_transmission(img, transmission, radius=60, eps=1e-3):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) / 255.0
    refined_transmission = cv2.ximgproc.guidedFilter(guide=gray_img, src=transmission, radius=radius, eps=eps)
    return refined_transmission

In [6]:
def recover_scene_radiance(img, transmission, atmospheric_light, t0=0.1):
    transmission = np.maximum(transmission, t0)
    radiance = (img - atmospheric_light) / transmission[:, :, None] + atmospheric_light
    return np.clip(radiance, 0, 1)

## Dehazing

In [7]:
def dehaze_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = np.array(img).astype(np.float32) / 255.0
    dark_channel_img = dark_channel(img,15)
    atmospheric_light = estimate_atmospheric_light(img, dark_channel_img)
    transmission = estimate_transmission(img, atmospheric_light)
    refined_transmission = refine_transmission(img, transmission)
    dehazed_img = recover_scene_radiance(img, refined_transmission, atmospheric_light)
    return (dehazed_img * 255).astype(np.uint8)


## Metrics Calculation

In [8]:
def calculate_ssim(ground_img,output_img):    
    ssim_value, ssim_map = ssim(ground_img, output_img, full=True,data_range=255)
    return ssim_value

def calculate_psnr(ground_img,output_img):
    psnr_value = psnr(ground_img, output_img, data_range=255)
    return psnr_value


## Model Output

In [21]:
def function_dehaze(hazy_images,clear_images,location):
    psnr_tot=0
    ssim_tot=0
    for hazy_path, clear_path in zip(hazy_images, clear_images):
    
        hazy_image = Image.open(hazy_path).convert("RGB")
        hazy_image=np.array(hazy_image).astype(np.float32) / 255.0
        clear_image = Image.open(clear_path).convert("RGB")
        clear_image=np.array(clear_image).astype(np.float32) / 255.0
    # print(hazy_path)
    
        dehazed_image=dehaze_image(hazy_path)
        filename = os.path.basename(hazy_path)
    # print(filename)

        psn=calculate_psnr(clear_image,dehazed_image)
        # print(psn)
        psnr_tot+=psn

        ssm=calculate_ssim(clear_image,dehazed_image)
        ssim_tot+=ssm
    
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))

        # Clear image
        axes[0].imshow(clear_image)
        axes[0].set_title("Clear Image")
        axes[0].axis("off")

        # Hazy image
        axes[1].imshow(hazy_image)
        axes[1].set_title("Hazy Image")
        axes[1].axis("off")

        # Dehazed image
        axes[2].imshow(dehazed_image)
        axes[2].set_title("Dehazed Image")
        axes[2].axis("off")

        output_filename = f'{location}/{os.path.splitext(filename)[0]}.png'
        plt.tight_layout()
        plt.savefig(f'{output_filename}')
        plt.close(fig)
    return psnr_tot,ssim_tot

## NH-Haze datasets

In [12]:
nhaze = "../Data/NH-HAZE"  

In [13]:
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff"}

def list_images(folder_path):
    files = sorted([
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, f)) and os.path.splitext(f)[1].lower() in image_extensions
    ])
    return files


total_images = list_images(nhaze)
# clear_images = list_images(clear_folder)

In [14]:
def calculate_ssim(ground_img, output_img):
    
    ground_img = (ground_img - ground_img.min()) / (ground_img.max() - ground_img.min())
    output_img = (output_img - output_img.min()) / (output_img.max() - output_img.min())

    if ground_img.shape != output_img.shape:
        output_img = cv2.resize(output_img, (ground_img.shape[1], ground_img.shape[0]))
    min_dim = min(ground_img.shape[:2])
    win_size = 3 

    if win_size % 2 == 0:
        win_size -= 1

    ssim_value, _ = ssim(ground_img, output_img, full=True, data_range=1.0, win_size=3)
    return ssim_value

In [15]:
hazy_images=[]
clear_images=[]
for i in range(len(total_images)):
    if(i%2==0):
        clear_images.append(total_images[i])
    else:
        hazy_images.append(total_images[i])

In [22]:
nh_psnr,nh_ssim=function_dehaze(hazy_images,clear_images,'Haze')

In [18]:
print(nh_psnr/55,' ',nh_ssim/55)

9.775522856999487   0.2787305084815233


## sots-indoor


In [29]:
clear_folder = "../Data/SOTS/indoor/gt"  
hazy_folder =  "../Data/SOTS/indoor/hazy"  

In [30]:
clear_images=list_images(clear_folder)

In [33]:
total_hazy_images=list_images(hazy_folder)

In [38]:
hazy_images=[]
for i in range(len(total_hazy_images)):
    if i%5==0 and i%10!=0:
        hazy_images.append(total_hazy_images[i])

In [40]:
soti_psnr,soti_ssim=function_dehaze(hazy_images,clear_images,'sots_indoor')